In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen, HTTPError
import csv

## Save working links for scores to CSV

In [2]:
next_year_dict = {}
for year in range(4, 19):
    year_str = '0'+str(year) if year<10 else str(year)
    next_year = year + 1
    next_year_str = '0'+str(next_year) if next_year<10 else str(next_year)
    next_year_dict[year_str] = next_year_str

In [3]:
def parse_link(template, verbose=False):
    try:
        html = urlopen(template)
        if verbose:
            print('sucess:', template)
        return html
    except HTTPError as e:
        if verbose:
            print('failure:', template)
        return None

In [4]:
gp_templates = [    
    'http://www.isuresults.com/results/gp{0}{1}/CAT00{3}RS.HTM',
    'http://www.isuresults.com/results/gp{0}20{1}/CAT00{3}RS.HTM',
    'http://www.isuresults.com/results/gp{0}{1}{2}/CAT00{3}RS.HTM',
    'http://www.isuresults.com/results/season{1}{2}/gp{0}{1}{2}/CAT00{3}RS.HTM',
    'http://www.isuresults.com/results/season{1}{2}/gp{0}{1}{2}/data0{3}90.htm',
    'http://www.isuresults.com/results/season{1}{2}/gp{0}20{1}/CAT00{3}RS.HTM',
    'http://www.isuresults.com/results/season{1}{2}/gp{0}20{1}/data0{3}90.htm'      
]
cp_templates = [
    'http://www.isuresults.com/results/{0}20{2}/CAT00{3}RS.HTM',
    'http://www.isuresults.com/results/season{1}{2}/{0}20{2}/CAT00{3}RS.HTM'
]

In [5]:
gp_events = ['usa', 'can', 'fra', 'rus', 'chn', 'jpn', 'fin', 'f']
cp_events = ['ec', 'fc', 'owg', 'wc']
events = gp_events + cp_events
event_new_names = ['US', 'CA', 'FR', 'RU', 'CN', 'JP', 'FI', 'FN', 'EU', '4C', 'OL', 'WR']
event_name_dict = dict(zip(events, event_new_names))
event_name_dict

{'usa': 'US',
 'can': 'CA',
 'fra': 'FR',
 'rus': 'RU',
 'chn': 'CN',
 'jpn': 'JP',
 'fin': 'FI',
 'f': 'FN',
 'ec': 'EU',
 'fc': '4C',
 'owg': 'OL',
 'wc': 'WR'}

In [545]:
with open('links/male.csv', mode='w') as file:    
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['event', 'year', 'url'])
    for event in events:
        for year_str, next_year_str in next_year_dict.items():
            if (next_year_str == '12' and event == 'wc') or (year_str in ['08', '09'] and event == 'f'):
                gender = '5'
            elif next_year_str in ['11', '12'] and event in ['ec', 'wc']:
                gender = '4'
            else:
                gender = '1'
            templates = gp_templates if event in gp_events else cp_templates
            for template in templates:
                template = template.format(event, year_str, next_year_str, gender)
                html = parse_link(template)
                if html is not None:
                    writer.writerow([event_name_dict[event], int('20'+next_year_str), template])
                    break

In [546]:
with open('links/female.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['event', 'year', 'url'])
    for event in events:
        for year_str, next_year_str in next_year_dict.items():
            if (next_year_str == '12' and event == 'wc') or (year_str in ['08', '09'] and event == 'f'):
                gender = '6'
            elif next_year_str in ['11', '12'] and event in ['ec', 'wc']:
                gender = '5'
            else:
                gender = '2'      
            templates = gp_templates if event in gp_events else cp_templates
            for template in templates:
                template = template.format(event, year_str, next_year_str, gender)
                html = parse_link(template)
                if html is not None:
                    writer.writerow([event_name_dict[event], int('20'+next_year_str), template])
                    break

## Parse score from saved links

In [6]:
male_links = pd.read_csv('links/male.csv')
male_links = pd.pivot(values=male_links['url'], index=male_links['event'], columns=male_links['year'])

In [7]:
female_links = pd.read_csv('links/female.csv')
female_links = pd.pivot(values=female_links['url'], index=female_links['event'], columns=female_links['year'])

In [17]:
male_counts = pd.read_csv('counts/male.csv', index_col=0)
male_counts

,US,CA,FR,RU,CN,FI,JP,FN,EU,4C,OL,WR
2005,11,12,11,10,10,NaN,12,0,25,23,0,23
2006,12,11,11,12,12,NaN,11,0,24,24,24,24
2007,11,11,11,12,10,NaN,11,0,24,21,0,24
2008,12,12,11,12,12,NaN,12,0,25,21,0,24
2009,10,11,12,12,11,NaN,12,0,24,24,0,24
2010,12,12,12,11,12,NaN,12,0,20,20,24,24
2011,12,12,9,12,11,NaN,12,0,24,20,0,24
2012,10,10,9,9,8,NaN,9,0,23,24,0,24
2013,10,10,8,9,7,NaN,9,0,22,23,0,24
2014,8,10,8,8,9,NaN,9,0,24,24,24,23


In [15]:
year = 2005
for event in event_name_dict.values():
    link = male_links.loc[event, year]
    
    if not pd.isnull(link):
        print(f'{event}: {link}')
        score = parse_score(link, event, year, n_skaters=10, process=True)
        print(score)

US: http://www.isuresults.com/results/gpusa04/CAT001RS.HTM
name_col: 1, score_col: 3, start_row: 1
                   name   score event  year
1         Brian JOUBERT  193.46    US  2005
2           Ryan JAHNKE  186.71    US  2005
3         Michael WEISS  179.56    US  2005
4           Roman SEROV  174.42    US  2005
5          Evan LYSACEK  162.51    US  2005
6              Lun SONG  161.46    US  2005
7        Nicholas YOUNG  160.82    US  2005
8          Ben FERREIRA  157.41    US  2005
9      Stefan LINDEMANN  156.73    US  2005
10  Vakhtang MURVANIDZE  143.10    US  2005
CA: http://www.isuresults.com/results/gpcan04/CAT001RS.HTM
name_col: 1, score_col: 3, start_row: 3
                    name   score event  year
9         Emanuel SANDHU  204.17    CA  2005
10          Ben FERREIRA  200.46    CA  2005
11        Jeffrey BUTTLE  191.85    CA  2005
12           Ryan JAHNKE  190.55    CA  2005
13  Kevin VAN DER PERREN  182.46    CA  2005
14      Stefan LINDEMANN  176.59    CA  2005
15 

In [13]:
def parse_score(link, event, year, n_skaters, process=True):
    html = urlopen(link)
    try:
        bs = BeautifulSoup(html.read(), 'html.parser')

        table_str = str(bs.findAll('table')[0])
        table = pd.read_html(table_str)[0]
        if process:
            if year == 2005 and event == 'US':
                name_col, score_col, start_row = 1, 3, 1
            elif (year in [2018, 2019] and event == 'JP') or (year == '2018' and event == 'FN'):
                name_col, score_col, start_row = 1, 5, 1
            elif year in range(2005, 2008):
                name_col, score_col, start_row = 1, 3, 3
            elif year in range(2008, 2020):
                name_col, score_col, start_row = 1, 6, 2
            print(f'name_col: {name_col}, score_col: {score_col}, start_row: {start_row}')
            table = table[[name_col, score_col]].dropna().iloc[start_row:start_row+n_skaters, :]
            table.columns = ['name', 'score']
            table['score'] = table['score'].astype(float)
            table['event'] = event            
            table['year'] = year
        return table
    except Exception as e:
        print('failure:', link)
        print(e)
        return None

,name,score,event,year


In [214]:
for url in male_links['url']:
    parse_score(link, event_name, year)

0      http://www.isuresults.com/results/gpusa05/CAT0...
1      http://www.isuresults.com/results/gpusa06/CAT0...
2      http://www.isuresults.com/results/gpusa07/CAT0...
3      http://www.isuresults.com/results/gpusa08/CAT0...
4      http://www.isuresults.com/results/gpusa09/CAT0...
5      http://www.isuresults.com/results/gpusa2010/CA...
6      http://www.isuresults.com/results/gpusa2011/CA...
7      http://www.isuresults.com/results/gpusa2012/CA...
8      http://www.isuresults.com/results/gpusa2013/CA...
9      http://www.isuresults.com/results/gpusa2014/CA...
10     http://www.isuresults.com/results/season1516/g...
11     http://www.isuresults.com/results/season1617/g...
12     http://www.isuresults.com/results/season1718/g...
13     http://www.isuresults.com/results/gpcan05/CAT0...
14     http://www.isuresults.com/results/gpcan06/CAT0...
15     http://www.isuresults.com/results/gpcan07/CAT0...
16     http://www.isuresults.com/results/gpcan08/CAT0...
17     http://www.isuresults.co

In [58]:
scores = pd.concat((parse_score(link, event_name, 2017) for event_name, link in links.items()), axis=0)
scores['name'] = scores['name'].str.replace('\xa0', ' ')
scores.reset_index(drop=True, inplace=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(scores)

                      name   score event  year
0              Nathan CHEN  275.88    US  2017
1              Adam RIPPON  266.45    US  2017
2           Sergei VORONOV  257.49    US  2017
3               Boyang JIN  246.03    US  2017
4                  Han YAN  228.33    US  2017
5               Ross MINER  219.62    US  2017
6            Takahito MURA  212.77    US  2017
7               Liam FIRUS  210.83    US  2017
8           Kevin REYNOLDS  204.05    US  2017
9           Roman SADOVSKY  200.10    US  2017
10               Shoma UNO  301.10    CA  2017
11             Jason BROWN  261.14    CA  2017
12       Alexander SAMARIN  250.06    CA  2017
13            Patrick CHAN  245.70    CA  2017
14         Jorik HENDRICKX  237.31    CA  2017
15          Michal BREZINA  237.04    CA  2017
16          Nicolas NADEAU  229.43    CA  2017
17          Keegan MESSING  217.75    CA  2017
18            Jun Hwan CHA  210.32    CA  2017
19              Paul FENTZ  201.60    CA  2017
20           

In [61]:
scores.to_csv('scores/2017.csv', index=False)